<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/data-augmentation/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 as cv
import os
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/"

ModuleNotFoundError: No module named 'google.colab'

In [2]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\facial-emotion-recognition')
LABELS = os.listdir(PATH)

In [3]:
count = []
for label in LABELS:
  dir = f'{PATH}/{label}'
  folder_item_count = len(os.listdir(dir))
  count.append(folder_item_count)
  print(f'{label}: {folder_item_count}')

HIGHEST_COUNT = max(count) + 1094
print(f'Highest count: {HIGHEST_COUNT}')

angry: 890
disgust: 439
fear: 570
happy: 1406
neutral: 524
sad: 746
surprise: 775
Highest count: 2500


In [4]:
def rotate(img):
  Cx, Cy = img.shape[:2]
  rand_angle = random.randint(-180,180)
  M = cv.getRotationMatrix2D((Cy//2, Cx//2),rand_angle ,1)
  img = cv.warpAffine(img, M, (Cx, Cy))
  return img

def flip(img):
  num = [0, 1, -1]
  img = cv.flip(img, random.choice(num))
  return img

def noise(img):
  row, col = img.shape[:2]
  number_of_pixels = random.randint(300, 1000)
  for i in range(number_of_pixels):
    y_coord=random.randint(0, row - 1)
    x_coord=random.randint(0, col - 1)
    img[y_coord][x_coord] = 255

  number_of_pixels = random.randint(300 , 1000)
  for i in range(number_of_pixels):
      y_coord=random.randint(0, row - 1)
      x_coord=random.randint(0, col - 1)
      img[y_coord][x_coord] = 0
  return img

In [5]:
data = {}

for i, label in enumerate(LABELS):
  path = f'{PATH}/{label}'
  data[label] = []
  for j, img_path in enumerate(os.listdir(path)):
    img = cv.imread(f'{path}/{img_path}')
    data[label].append(img)

In [9]:
for label in LABELS:
  images = data[label]

  path = f'C:\\Users\\vinar\\Downloads\\facial-emotion-recognition-augmented\\{label}'
  os.makedirs(path, exist_ok=True)

  for j, img in enumerate(images):
    if(len(os.listdir(path)) >= HIGHEST_COUNT):
      break

    num = [1,2,3]
    rand = random.choice(num)

    for i in range(8):
      if(len(os.listdir(path)) >= HIGHEST_COUNT):
        break
      if(rand == 1):
        img = rotate(img)
      elif(rand == 2):
        img = flip(img)
      elif(rand == 3):
        img = noise(img)


      img_name = f'{path}/{label}_{j}_{i}.png'
      cv.imwrite(img_name, img)

In [3]:
count = []
for label in LABELS:
  dir = f'{PATH}/{label}'
  folder_item_count = len(os.listdir(dir))
  count.append(folder_item_count)
  print(f'{label}: {folder_item_count}')

LOWEST_COUNT = min(count)
print(f'Highest count: {LOWEST_COUNT}')

angry: 890
disgust: 439
fear: 570
happy: 1406
neutral: 524
sad: 746
surprise: 775
Highest count: 439


In [6]:
for label in LABELS:
  folder_dir = f'{PATH}/{label}'

  path = f'C:\\Users\\vinar\\Downloads\\down-sampling\\{label}'
  os.makedirs(path, exist_ok=True)

  for i, image in enumerate(os.listdir(folder_dir)):
    if(i == LOWEST_COUNT):
      break

    img = cv.imread(f'{folder_dir}/{image}')
    cv.imwrite(f'{path}/{image}', img)